# <a id='toc1_'></a>[The AI Idea Factory](#toc0_)
For my very first project in this year-long challenge, I'm gonna be creating a creative idea assitant. It will help me keep up with recent trends and also keep every new idea fresh!
Let's see how I've built it!

**Table of contents**<a id='toc0_'></a>    
- [The AI Idea Factory](#toc1_)    
  - [Step 0: Setting things up](#toc1_1_)    
  - [Step 1: Creating the agents](#toc1_2_)    
    - [Researcher Agent](#toc1_2_1_)    
    - [Creative Director](#toc1_2_2_)    
    - [Planner](#toc1_2_3_)    
    - [Writer](#toc1_2_4_)    
    - [Content Reviewer](#toc1_2_5_)    
    - [Project Manager](#toc1_2_6_)    
  - [Step 2: Tasks](#toc1_3_)    
    - [Research Task](#toc1_3_1_)    
    - [Ideation Task](#toc1_3_2_)    
    - [Planning Task](#toc1_3_3_)    
    - [Instruction Writng Task](#toc1_3_4_)    
    - [Blog Post Writing Task](#toc1_3_5_)    
    - [Blog Post Review Task](#toc1_3_6_)    
    - [Blog Post Finalization Task](#toc1_3_7_)    
    - [Project Management Task](#toc1_3_8_)    
  - [Crew](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Step 0: Setting things up](#toc0_)
In the first stage, we are going to be setting the foundation for this project, including setting up our LLMs, importing packages, writing our own custom tools etc.

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileWriterTool
from duckduckgo_search import DDGS
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = "NA" # The API key is required, but unused

For LLMs, I will be using llama3.1 for text generation and semantic thinking, while mistral will be used for calling tools since it excels at following instructions.

In [3]:
llama3p1 = ChatOpenAI(
    model="ollama/llama3.1", # Use a model that you have pulled
    base_url="http://localhost:11434/v1",
)
mistral = ChatOpenAI(
    model="mistral",
    base_url="http://localhost:11434/v1",
)

Here, I have created a DuckDuckGo search tool to give my agents the ability to search the web for new ideas and breakthrough technologies.

In [4]:
def internet_search(query: str) -> list:
    """Search Internet for relevant information based on a query. The input should **ONLY** be the query you want to search."""
    ddgs = DDGS()
    results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
    return str(results)

internet_search_tool = Tool(
    name="Internet Search",
    func=internet_search,
    description="Search the internet for relevant information based on a query."
)

Since the agents will have to come up with **new** ideas or suggest upgrades to previous projects, it is only natural to give them access to the all the previous projects that have been done before. I do that by simply reading all the ```.txt``` files in my ideas directory.

In [5]:
def read_directory(directory):
    content_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                content = file.read()
                content_list.append(content)
    return content_list

all_projects = read_directory('Generated-Blog-Posts/')

In [6]:
file_writer_tool = FileWriterTool()

## <a id='toc1_2_'></a>[Step 1: Creating the agents](#toc0_)
CrewAI has 3 main components: Agents, Tasks and Crews. Think of agents as colligues working together to achieve their final goal, while each person has their own set goals and skills.
In this project, we will create 6 Agents.


### <a id='toc1_2_1_'></a>[Researcher Agent](#toc0_)

In [7]:
researcher = Agent(
    role='AI Research Specialist',
    goal='Discover and summarize cutting-edge AI technologies and advancements',
    backstory="""You are a top-tier AI research specialist with a passion for staying at the forefront of technological advancements. Your expertise spans multiple domains within AI, and you have a knack for identifying promising new developments.
    To effectively research and summarize AI advancements:
    1. Start by identifying key areas of AI to focus on (e.g., NLP, computer vision, reinforcement learning).
    2. For each area:
       a. Search recent publications on arXiv and top AI conferences.
       b. Look for announcements from major AI companies and research labs.
       c. Analyze the potential impact and novelty of each finding.
    3. Synthesize your findings:
       a. What are the most significant breakthroughs?
       b. How might these advancements be applied to real-world problems?
       c. What are the potential implications for future AI development?
    4. Organize your research into a clear, concise report.
    5. Include citations and links to original sources.

    Remember to use a step-by-step approach (Chain of Thought) when analyzing complex information, and continuously reason about your actions and their outcomes (ReAct).
    """,
    verbose=True,
    allow_delegation=False,
    llm=llama3p1,
    tools=[internet_search_tool], 
)

### <a id='toc1_2_2_'></a>[Creative Director](#toc0_)

In [8]:
creative_director = Agent(
    role='AI Innovation Strategist',
    goal='Generate innovative and feasible AI project ideas',
    backstory="""You are a visionary AI strategist with a track record of conceptualizing groundbreaking projects. Your strength lies in identifying unique applications of AI that solve real-world problems.""",
    verbose=True,
    allow_delegation=False,
    llm=llama3p1,
    instructions=f"""
    To create compelling AI project ideas:
    1. Review the research findings provided by the AI Research Specialist.
    2. Reflect on past projects and identify areas for improvement or expansion.
    3. For each potential idea:
       a. Consider its novelty and potential impact.
       b. Evaluate feasibility given current technology and resources.
       c. Think about potential challenges and how they might be overcome.
    4. Use lateral thinking to combine different AI technologies in unique ways.
    5. Prioritize ideas based on:
       a. Alignment with project goals
       b. Technical feasibility
       c. Potential for innovation
    6. Develop a detailed outline for the top idea, including:
       a. Project objectives
       b. Key technologies involved
       c. High-level implementation steps
       d. Potential applications and benefits

    Employ Chain of Thought reasoning to break down complex problems, and use ReAct to continuously evaluate and refine your ideas.
    Remember, the projects shouldn't be too hard and time consuming. Aim for projects that can be done in a week in 10 to 12 hours.
    These are tha previous projects that have been done: 

    {'\n\n'.join(all_projects)}
    """
)


### <a id='toc1_2_3_'></a>[Planner](#toc0_)

In [9]:
planner = Agent(
    role='AI Project Architect',
    goal='Create detailed, actionable project plans for AI initiatives',
    backstory="""You are an experienced AI project manager with a deep understanding of software development lifecycles and AI implementation challenges. Your plans are known for their clarity and effectiveness.""",
    verbose=True,
    allow_delegation=False,
    llm=llama3p1,
    instructions="""
    To develop comprehensive project plans:
    1. Analyze the project outline provided by the AI Innovation Strategist.
    2. Break down the project into main phases (e.g., research, development, testing, deployment).
    3. For each phase:
       a. Identify key tasks and subtasks.
       b. Estimate time and resource requirements.
       c. Define clear deliverables and milestones.
    4. Consider potential risks and develop mitigation strategies.
    5. Create a timeline with dependencies between tasks.
    6. Allocate resources effectively, considering skill requirements for each task.
    7. Define success criteria and evaluation metrics for the project.
    8. Outline a communication and collaboration strategy for the team.

    Use Chain of Thought to systematically work through each aspect of the plan, and apply ReAct principles to anticipate and address potential issues proactively.
    Remember, the projects should be suitable for a one week time-frame. Aim for 10-12 hours.
    """,
    tools=[file_writer_tool]
)

### <a id='toc1_2_4_'></a>[Writer](#toc0_)

In [10]:
writer = Agent(
    role='AI Content Specialist',
    goal='Craft engaging and informative content about AI projects',
    backstory="""You are a talented writer with a deep understanding of AI technologies and a gift for explaining complex concepts in accessible language. Your content is known for its clarity and ability to inspire.""",
    verbose=True,
    allow_delegation=False,
    llm=llama3p1,
    instructions="""
    To create compelling content about AI projects:
    1. Review the project plan and key details provided.
    2. Identify the core message and unique selling points of the project.
    3. Structure your content:
       a. Craft an attention-grabbing headline.
       b. Write an engaging introduction that highlights the project's significance.
       c. Explain the AI technologies involved in simple terms.
       d. Describe the potential impact and applications of the project.
       e. Include any challenges overcome or innovative approaches used.
       f. Conclude with a call to action or future outlook.
    4. Use analogies and real-world examples to illustrate complex concepts.
    5. Incorporate relevant statistics or data to support key points.
    6. Ensure the tone is professional yet accessible to a broad audience.
    7. Optimize the content for the specific platform (e.g., LinkedIn post format).

    Apply Chain of Thought reasoning to organize your ideas coherently, and use ReAct to continuously refine your writing based on the project's goals and audience expectations.
    """,
    tools=[file_writer_tool]
)


### <a id='toc1_2_5_'></a>[Content Reviewer](#toc0_)

In [11]:
content_reviewer = Agent(
    role='AI Communication Quality Assurer',
    goal='Ensure the highest quality and accuracy of AI project communications',
    backstory="""You are a meticulous editor with extensive knowledge of AI technologies and best practices in technical communication. Your keen eye for detail ensures that all content is clear, accurate, and impactful.""",
    verbose=True,
    allow_delegation=False,
    llm=llama3p1,
    instructions="""
    To effectively review and enhance AI project content:
    1. Carefully read the entire piece of content.
    2. Assess the overall structure and flow of information.
    3. Verify technical accuracy:
       a. Check that AI concepts are correctly explained.
       b. Ensure consistency with the original project plan.
       c. Verify any statistics or data cited.
    4. Evaluate clarity and accessibility:
       a. Identify any jargon that needs simplification.
       b. Ensure complex ideas are adequately explained.
    5. Assess engagement and impact:
       a. Verify that the content captures the project's significance.
       b. Ensure the tone is appropriate for the target audience.
    6. Check for grammar, spelling, and punctuation errors.
    7. Provide constructive feedback:
       a. Highlight strengths of the content.
       b. Suggest specific improvements with explanations.
    8. If necessary, request revisions from the writer.

    Use Chain of Thought to systematically evaluate each aspect of the content, and apply ReAct principles to provide thoughtful, actionable feedback that improves the overall quality of the communication.
    """
)

### <a id='toc1_2_6_'></a>[Project Manager](#toc0_)

In [12]:
project_manager = Agent(
    role='AI Project Orchestrator',
    goal='Ensure smooth execution and coordination of the AI project development process',
    backstory="""You are a seasoned project manager with a track record of successfully delivering complex AI initiatives. Your strength lies in coordinating diverse teams and ensuring all aspects of a project come together seamlessly.""",
    verbose=True,
    allow_delegation=True,
    llm=llama3p1,
    instructions="""
    To effectively manage the AI project development process:
    1. Familiarize yourself with the overall project goal and the roles of each team member.
    2. Establish clear communication channels and protocols for the team.
    3. For each phase of the project:
       a. Ensure all team members understand their tasks and deadlines.
       b. Monitor progress and identify any potential bottlenecks.
       c. Facilitate communication between team members as needed.
       d. Adjust timelines or resources if necessary to keep the project on track.
    4. Regularly update all stakeholders on project status and any significant developments.
    5. Anticipate and proactively address potential challenges or conflicts.
    6. Ensure that deliverables from each team member meet quality standards and project requirements.
    7. Facilitate smooth transitions between different stages of the project (e.g., from planning to writing to review).
    8. Conduct a final review to ensure all project objectives have been met before concluding each weekly cycle.

    Utilize Chain of Thought reasoning to break down complex project management tasks, and apply ReAct principles to continuously assess and optimize the project's progress and team dynamics.
    """
)

## <a id='toc1_3_'></a>[Step 2: Tasks](#toc0_)
As said by the official documents of CrewAI, tasks are specific assignments completed by agents. They provide all necessary details for execution, such as a description, the agent responsible, required tools, and more, facilitating a wide range of action complexities.

Tasks within crewAI can be collaborative, requiring multiple agents to work together. This is managed through the task properties and orchestrated by the Crew's process, enhancing teamwork and efficiency.

### <a id='toc1_3_1_'></a>[Research Task](#toc0_)

In [13]:
research_task = Task(
    expected_output="A detailed article on the latest AI advancements and break-through technologies.",
    description="""You must use the internet_search_tool to search the web on the most recent AI and generative AI technologies, trends and hot topics.
    After you have done that, write an artivle about them with no loss of information containing but not limited to:
    how the technology works, its use cases, potential improvements etc.""",
    agent=researcher,
    delegations=False
)

### <a id='toc1_3_2_'></a>[Ideation Task](#toc0_)

In [14]:
ideation_task = Task(
    expected_output="A novel, creative and actionable idea which is about what we have done so far and also about the newest AI tech advancements.",
    description=f"""Based on the research findings and the context of previous projects, generate an 
    innovative AI project idea. Create a detailed project outline including:
    1. Project title
    2. Main objective
    3. Key AI technologies to be used
    4. Potential real-world applications
    5. Brief overview of implementation approach

    **Important**: The project should be easily done in one week, so don't complicate it. Aim for 10-12 hours in one week.
    
    Context: {all_projects}""",
    agent=creative_director,
    delegations=False
)

### <a id='toc1_3_3_'></a>[Planning Task](#toc0_)

In [15]:
planning_task = Task(
    expected_output="A detailed step by step plan on how to achieve the project that the creative director has come up with. Organize The tasks into small, actionable steps.",
    description="""Take the project idea and create a detailed implementation plan. Break down the 
    project into specific steps, estimate time requirements, identify potential challenges, and 
    suggest solutions. Your plan should be actionable and clear enough for a developer to follow. 
    **Important**: The project should be easily done in one week, so don't complicate it. Aim for 10-12 hours in one week.
    """,
    agent=planner,
    delegations=False
)

### <a id='toc1_3_4_'></a>[Instruction Writng Task](#toc0_)

In [16]:
write_instructions_task = Task(
    expected_output="A text file in the Generated-Planned-Project directory containing an easy to follow guide/project outline based on the steps you have come up with.",
    description="""Based on your detailed plan, write clear and concise instructions for implementing 
    the project. These instructions should guide a developer through the process of building the 
    project. Save these instructions to a file named 'project_instructions.md' in the 'Generated-Planned-Project' directory.
    **Important**: The project should be easily done in one week, so don't complicate it. Aim for 10-12 hours in one week.
    """,
    agent=planner,
    delegations=False,
)

### <a id='toc1_3_5_'></a>[Blog Post Writing Task](#toc0_)

In [17]:
write_blog_post_task = Task(
    expected_output="A professional and engaging LinkedIn post.",
    description="""Create an engaging blog post about the AI project. The post should include:
    1. An attention-grabbing title
    2. A brief introduction to the project and its significance
    3. An explanation of the AI technologies used
    4. The potential impact and applications of the project
    5. Any challenges overcome or innovative approaches used
    6. A conclusion with future implications or a call to action
    7. A good amount of suitable emojis
    
    Aim for a word count between 500-750 words.""",
    agent=writer,
    delegations=False
)

### <a id='toc1_3_6_'></a>[Blog Post Review Task](#toc0_)

In [18]:
review_post_task = Task(
    expected_output="A thorough review of the blog post made by the writer, containing actionable improvements.",
    description="""Review the blog post created by the writer. Ensure it is engaging, informative, 
    and technically accurate. Check for clarity, proper structure, and appropriate tone. Provide 
    specific feedback for improvements if necessary.""",
    agent=content_reviewer,
    delegations=False
)

### <a id='toc1_3_7_'></a>[Blog Post Finalization Task](#toc0_)

In [19]:
finalize_post_task = Task(
    expected_output="A markdown file in the Generated-Blog-Posts directory containing a sophisticated and professionl post suitable for LinkedIn",
    description="""Incorporate the feedback from the content reviewer to finalize the blog post. 
    Make necessary edits to improve clarity, engagement, and technical accuracy. Save the final 
    version as 'final_weekly_project_post.md' in the 'Generated-Blog-Posts' directory.""",
    agent=writer,
    delegations=False,
)

### <a id='toc1_3_8_'></a>[Project Management Task](#toc0_)

In [20]:
project_management_task = Task(
    expected_output="Oversee and manage the whole project and give me a bried rundown of all that has happened.",
    description="""Oversee the entire process of generating the AI project idea, planning, and 
    content creation. Ensure all tasks are completed in the correct order and that the output of 
    each task is suitable for the next. Coordinate between agents if clarification or additional 
    information is needed. Provide a final summary report of the week's activities and outcomes.""",
    delegations=False
)

## <a id='toc1_4_'></a>[Crew](#toc0_)
The last essential part to any CrewAI program is the crew. Think of the crew as a workspace, where all the collegues work together and collaborate.

In [21]:
crew = Crew(
    agents=[researcher, creative_director, planner, writer, content_reviewer],
    tasks=[research_task, ideation_task, planning_task, write_instructions_task, write_blog_post_task, review_post_task, finalize_post_task, project_management_task],
    verbose=True,
    manager_agent=project_manager,
    manager_llm=llama3p1,
    process=Process.hierarchical,
)

In [ ]:
crew.kickoff()